# Linear Regression Model

In [60]:
import kagglehub
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

# Regression Classifier
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.metrics import mean_squared_error

from itertools import combinations
import pandas as pd

In [61]:
# Load dataset.
# Download latest version
path = kagglehub.dataset_download("lashagoch/life-expectancy-who-updated")
print("Path to dataset files:", path)

file = "Life-Expectancy-Data-Updated.csv"

data = pd.read_csv(path + "/" + file)

# FEATURE ENGINEERING
# take log of GDP per capita
data["GDP_per_capita"] = np.log10(data["GDP_per_capita"])

# take average vaccination percentage
data["Vaccination_score"] = (data["Hepatitis_B"] + data["Polio"] + data["Diphtheria"])/3

# Compute Lifestyle Index
BMI_score = 1*(data["BMI"] <= 30)*(data["BMI"] > 25) + 2*(data["BMI"] > 30)
Alcohol_score = 1*(data["Alcohol_consumption"] <= 9.722)*(data["Alcohol_consumption"] > 3.241) + 2*(data["Alcohol_consumption"] > 9.722)
data["Lifestyle_index"] = BMI_score + Alcohol_score

X = data[['Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed', 'Vaccination_score', 'Lifestyle_index']]

y = data['Life_expectancy']

# Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Path to dataset files: /Users/nathanlonghurst/.cache/kagglehub/datasets/lashagoch/life-expectancy-who-updated/versions/1


In [ ]:
# list the columns of the data that we use to model
independent_columns = ['Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed']

best_bic = np.inf
best_aic = np.inf

# loop through all the combinations of features and find the combinations with best aic and bic
for num_features in range(3, len(independent_columns) + 1):
    for combo in combinations(independent_columns, r=num_features):
        temp_X = X_train[list(combo)]
        X = sm.add_constant(temp_X)
        model = sm.OLS(y_train, X).fit()
        if model.aic < best_aic:
            best_aic = model.aic
            best_aic_features = list(combo)
        if model.bic < best_bic:
            best_bic = model.bic
            best_bic_features = list(combo)


model_aic = sm.OLS(y_train, X_train[best_aic_features]).fit()
model_bic = sm.OLS(y_train, X_train[best_bic_features]).fit()

In [63]:
y_hat = model_aic.predict(X_test[best_aic_features])
print(f"mean squared error for best aic model: {mean_squared_error(y_test, y_hat)}")

y_hat = model_bic.predict(X_test[best_bic_features])
print(f"mean squared error for best bic model: {mean_squared_error(y_test, y_hat)}")

mean squared error for best aic model: 14.380734738619442
mean squared error for best bic model: 15.299104911461335


In [64]:
print(f"Best aic model had {len(best_aic_features)} features")
print(f"Best bic model had {len(best_bic_features)} features")

print(f"Best aic features: {best_aic_features}")
print(f"Best bic features: {best_bic_features}")

Best aic model had 12 features
Best bic model had 8 features
Best aic features: ['Alcohol_consumption', 'Hepatitis_B', 'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita', 'Population_mln', 'Thinness_ten_nineteen_years', 'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed']
Best bic features: ['Alcohol_consumption', 'BMI', 'Polio', 'Incidents_HIV', 'GDP_per_capita', 'Population_mln', 'Schooling', 'Economy_status_Developed']


In [65]:
model_aic.save("LinearRegressionAIC.pickle")
model_bic.save("LinearRegressionBIC.pickle")

In [66]:
print("Most important features best AIC score model")
model_aic.pvalues.sort_values(ascending=True)

Most important features best AIC score model


Incidents_HIV                  3.356532e-310
BMI                            7.169879e-214
GDP_per_capita                 2.672762e-176
Polio                           2.448268e-14
Economy_status_Developed        8.620342e-10
Population_mln                  8.286127e-09
Thinness_five_nine_years        7.942395e-08
Diphtheria                      5.482395e-05
Alcohol_consumption             6.485047e-05
Schooling                       2.515934e-03
Hepatitis_B                     1.014619e-02
Thinness_ten_nineteen_years     8.954088e-01
dtype: float64

In [67]:
print("Most important features best BIC score model")
model_bic.pvalues.sort_values(ascending=True)

Most important features best BIC score model


Incidents_HIV               5.504714e-279
BMI                         5.421786e-207
GDP_per_capita              2.154282e-184
Polio                       2.871759e-184
Population_mln               1.220101e-21
Alcohol_consumption          2.433836e-07
Economy_status_Developed     3.742900e-07
Schooling                    1.539289e-01
dtype: float64